In [243]:
import re
import json
from io import StringIO
from bs4 import BeautifulSoup
import requests
import csv

#Build Yahoo Finance URL's

In [244]:
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [245]:
stock = 'NVDA'

# Match embedded jscript url with regex to parse json data

In [246]:
response = requests.get(url_financials.format(stock, stock))

In [247]:
soup = BeautifulSoup(response.text, 'html.parser')

In [248]:
pattern = re.compile('\s--\sData\s--\s')
data = soup.find('script', text = pattern).contents[0]

In [ ]:
data[:111]

In [ ]:
data[:-11]

In [ ]:
start = data.find('context')-2
json_data = json.loads(data[start:-12])
json_data['context'].keys()

In [252]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [253]:
annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

In [272]:
quarterly_cf[0].keys()

dict_keys(['investments', 'changeToLiabilities', 'totalCashflowsFromInvestingActivities', 'netBorrowings', 'totalCashFromFinancingActivities', 'changeToOperatingActivities', 'issuanceOfStock', 'netIncome', 'changeInCash', 'endDate', 'repurchaseOfStock', 'totalCashFromOperatingActivities', 'depreciation', 'dividendsPaid', 'changeToInventory', 'changeToAccountReceivables', 'otherCashflowsFromFinancingActivities', 'maxAge', 'changeToNetincome', 'capitalExpenditures'])

# Annual Balance Sheet

In [255]:
parsed_statements = []
for s in annual_bs:
  statement = {}
  for k,v in s.items():
    try:
      statement[k] = v['raw']
    except TypeError:
      continue
    except KeyError:
      continue
    parsed_statements.append(statement)
print(parsed_statements[:11])

[{'intangibleAssets': 2737000000, 'capitalSurplus': 8721000000, 'totalLiab': 11898000000, 'totalStockholderEquity': 16893000000, 'otherCurrentLiab': 1207000000, 'totalAssets': 28791000000, 'endDate': 1612051200, 'commonStock': 1000000, 'retainedEarnings': 18908000000, 'otherLiab': 1375000000, 'goodWill': 4193000000, 'treasuryStock': -10737000000, 'otherAssets': 2806000000, 'cash': 847000000, 'totalCurrentLiabilities': 3925000000, 'deferredLongTermAssetCharges': 806000000, 'shortLongTermDebt': 999000000, 'otherStockholderEquity': 19000000, 'propertyPlantEquipment': 2856000000, 'totalCurrentAssets': 16055000000, 'longTermInvestments': 144000000, 'netTangibleAssets': 9963000000, 'shortTermInvestments': 10714000000, 'netReceivables': 2429000000, 'longTermDebt': 5964000000, 'inventory': 1826000000, 'accountsPayable': 1201000000}, {'intangibleAssets': 2737000000, 'capitalSurplus': 8721000000, 'totalLiab': 11898000000, 'totalStockholderEquity': 16893000000, 'otherCurrentLiab': 1207000000, 'to

# Quarterly Cash Flow

In [256]:
parsed_statements = []
for s in quarterly_cf:
  statement = {}
  for k,v in s.items():
    try:
      statement[k] = v['raw']
    except TypeError:
      continue
    except KeyError:
      continue
    parsed_statements.append(statement)
print(parsed_statements[:11])

[{'investments': -2846000000, 'changeToLiabilities': 74000000, 'totalCashflowsFromInvestingActivities': -3129000000, 'netBorrowings': -3000000, 'totalCashFromFinancingActivities': -342000000, 'changeToOperatingActivities': 232000000, 'issuanceOfStock': 4000000, 'netIncome': 1457000000, 'changeInCash': -1404000000, 'endDate': 1612051200, 'repurchaseOfStock': -226000000, 'totalCashFromOperatingActivities': 2067000000, 'depreciation': 288000000, 'dividendsPaid': -99000000, 'changeToInventory': -334000000, 'changeToAccountReceivables': 117000000, 'otherCashflowsFromFinancingActivities': -18000000, 'changeToNetincome': 233000000, 'capitalExpenditures': -283000000}, {'investments': -2846000000, 'changeToLiabilities': 74000000, 'totalCashflowsFromInvestingActivities': -3129000000, 'netBorrowings': -3000000, 'totalCashFromFinancingActivities': -342000000, 'changeToOperatingActivities': 232000000, 'issuanceOfStock': 4000000, 'netIncome': 1457000000, 'changeInCash': -1404000000, 'endDate': 16120

#Quarterly Income Statement

In [257]:
parsed_statements = []
for s in quarterly_is:
  statement = {}
  for k,v in s.items():
    try:
      statement[k] = v['raw']
    except TypeError:
      continue
    except KeyError:
      continue
    parsed_statements.append(statement)
print(parsed_statements[:11])

[{'researchDevelopment': 1146000000, 'incomeBeforeTax': 1470000000, 'netIncome': 1457000000, 'sellingGeneralAdministrative': 502000000, 'grossProfit': 3156000000, 'ebit': 1508000000, 'endDate': 1612051200, 'operatingIncome': 1508000000, 'interestExpense': -53000000, 'incomeTaxExpense': 13000000, 'totalRevenue': 5003000000, 'totalOperatingExpenses': 3495000000, 'costOfRevenue': 1847000000, 'totalOtherIncomeExpenseNet': -38000000, 'netIncomeFromContinuingOps': 1457000000, 'netIncomeApplicableToCommonShares': 1457000000}, {'researchDevelopment': 1146000000, 'incomeBeforeTax': 1470000000, 'netIncome': 1457000000, 'sellingGeneralAdministrative': 502000000, 'grossProfit': 3156000000, 'ebit': 1508000000, 'endDate': 1612051200, 'operatingIncome': 1508000000, 'interestExpense': -53000000, 'incomeTaxExpense': 13000000, 'totalRevenue': 5003000000, 'totalOperatingExpenses': 3495000000, 'costOfRevenue': 1847000000, 'totalOtherIncomeExpenseNet': -38000000, 'netIncomeFromContinuingOps': 1457000000, '

# Profile Data

In [258]:
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
response = requests.get(url_profile.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile('\s--\sData\s--\s')
data = soup.find('script', text = pattern).contents[0]
start = data.find('context')-2
json_data = json.loads(data[start:-12])

In [259]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'price', 'secFilings', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'assetProfile', 'pageViews'])

# Company Officers

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['companyOfficers']

# SEC Filings

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['secFilings'].keys()
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['secFilings']['filings']

# Business Summary

In [262]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['longBusinessSummary']

"NVIDIA Corporation operates as a visual computing company worldwide. It operates in two segments, GPU and Tegra Processor. The GPU segment offers processors, which include GeForce for PC gaming and mainstream PCs; GeForce NOW for cloud-based gaming; Quadro for design professionals working in computer-aided design, video editing, special effects, and other creative applications; Tesla for artificial intelligence (AI) utilizing deep learning, accelerated computing, and general purpose computing; GRID, which provides power of NVIDIA graphics through the cloud and datacenters; DGX for AI scientists, researchers, and developers; and EGX for accelerated AI computing at the edge. The Tegra Processor segment provides processors comprising SHIELD devices and services designed to harness the power of mobile-cloud to revolutionize home entertainment, AI, and gaming; AGX, a power-efficient AI computing platform for intelligent edge devices; DRIVE AGX for self-driving vehicles; Clara AGX for medic

In [263]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']

{'algorithm': None,
 'ask': {'fmt': '0.00', 'raw': 0},
 'askSize': {'fmt': '900', 'longFmt': '900', 'raw': 900},
 'averageDailyVolume10Day': {'fmt': '10.86M',
  'longFmt': '10,858,466',
  'raw': 10858466},
 'averageVolume': {'fmt': '7.86M', 'longFmt': '7,856,096', 'raw': 7856096},
 'averageVolume10days': {'fmt': '10.86M',
  'longFmt': '10,858,466',
  'raw': 10858466},
 'beta': {'fmt': '1.41', 'raw': 1.412242},
 'bid': {'fmt': '0.00', 'raw': 0},
 'bidSize': {'fmt': '900', 'longFmt': '900', 'raw': 900},
 'circulatingSupply': {},
 'currency': 'USD',
 'dayHigh': {'fmt': '502.00', 'raw': 502},
 'dayLow': {'fmt': '467.17', 'raw': 467.17},
 'dividendRate': {'fmt': '0.64', 'raw': 0.64},
 'dividendYield': {'fmt': '0.13%', 'raw': 0.0013},
 'exDividendDate': {'fmt': '2021-03-09', 'raw': 1615248000},
 'expireDate': {},
 'fiftyDayAverage': {'fmt': '552.93', 'raw': 552.93427},
 'fiftyTwoWeekHigh': {'fmt': '614.90', 'raw': 614.9},
 'fiftyTwoWeekLow': {'fmt': '180.68', 'raw': 180.68},
 'fiveYearAvgDiv

#Statistics


In [ ]:
url_profile = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
response = requests.get(url_profile.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile('\s--\sData\s--\s')
data = soup.find('script', text = pattern).contents[0]
start = data.find('context')-2
json_data = json.loads(data[start:-12])
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']

#Historical Data

In [278]:
download_url = 'https://query1.finance.yahoo.com/v7/finance/download/NVDA?period1=1583649152&period2=1615185152&interval=1d&events=history&includeAdjustedClose=true'
response = requests.get(download_url)
response.text[:1111]

'Date,Open,High,Low,Close,Adj Close,Volume\n2020-03-09,239.899994,259.399994,238.179993,245.440002,245.182236,15354000\n2020-03-10,256.540009,261.079987,245.990005,261.079987,260.805786,14516200\n2020-03-11,255.460007,256.600006,242.570007,246.470001,246.211151,12499100\n2020-03-12,225.460007,236.119995,215.800003,216.309998,216.082809,20413500\n2020-03-13,230.500000,241.020004,219.610001,240.839996,240.587051,15870900\n2020-03-16,212.410004,226.100006,194.000000,196.399994,196.193710,18174300\n2020-03-17,200.910004,220.860001,191.000000,217.270004,217.041824,20840800\n2020-03-18,200.100006,209.940002,180.679993,202.820007,202.606995,21856700\n2020-03-19,201.960007,216.729996,193.389999,212.970001,212.746323,19137800\n2020-03-20,219.000000,230.320007,205.130005,205.750000,205.533920,20115900\n2020-03-23,205.710007,216.479996,198.520004,212.690002,212.466629,16096900\n2020-03-24,229.000000,252.660004,228.050003,249.179993,248.918304,27541100\n2020-03-25,253.800003,261.820007,243.660004,

In [266]:
download_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
params = {
    'period1':'1583649152',
    'period2':'1615185152',
    'interval':'1d',
    'events':'history'
}

In [267]:
params = {
    'range':'5y',
    'interval':'1d',
    'events':'history'
}

In [268]:
response = requests.get(download_url.format(stock), params=params)

In [279]:
response.text[:1111]

'Date,Open,High,Low,Close,Adj Close,Volume\n2020-03-09,239.899994,259.399994,238.179993,245.440002,245.182236,15354000\n2020-03-10,256.540009,261.079987,245.990005,261.079987,260.805786,14516200\n2020-03-11,255.460007,256.600006,242.570007,246.470001,246.211151,12499100\n2020-03-12,225.460007,236.119995,215.800003,216.309998,216.082809,20413500\n2020-03-13,230.500000,241.020004,219.610001,240.839996,240.587051,15870900\n2020-03-16,212.410004,226.100006,194.000000,196.399994,196.193710,18174300\n2020-03-17,200.910004,220.860001,191.000000,217.270004,217.041824,20840800\n2020-03-18,200.100006,209.940002,180.679993,202.820007,202.606995,21856700\n2020-03-19,201.960007,216.729996,193.389999,212.970001,212.746323,19137800\n2020-03-20,219.000000,230.320007,205.130005,205.750000,205.533920,20115900\n2020-03-23,205.710007,216.479996,198.520004,212.690002,212.466629,16096900\n2020-03-24,229.000000,252.660004,228.050003,249.179993,248.918304,27541100\n2020-03-25,253.800003,261.820007,243.660004,

In [286]:
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)
for row in data[:5]:
  print(row)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2016-03-07', '32.369999', '32.590000', '31.920000', '32.340000', '31.758257', '5999200']
['2016-03-08', '32.009998', '32.200001', '31.650000', '31.750000', '31.178871', '6873500']
['2016-03-09', '32.029999', '32.080002', '31.320000', '31.730000', '31.159229', '5569700']
['2016-03-10', '31.920000', '32.099998', '31.040001', '31.680000', '31.110134', '7152100']
